#### Preamble for `CoLab` 

To use this notebook (if you haven't already) you can first save a copy to your local drive by clicking `File > Save a Copy in Drive` and run on that copy.

_Note_: `Colab` is a really handy way to test and try `strauss`, though it will generally run and display audio more slowly than running on your local machine. For a more responsive experience, why not install `strauss` locally, following the instructions [on the Github](https://github.com/james-trayford/strauss)

Run these cells, so that the notebook functions on the _Google_ `Colab` platform:

In [ ]:
%pip --quiet install strauss

In [ ]:
!git clone https://github.com/james-trayford/strauss.git

In [ ]:
%cd strauss/examples/

## <u> Demonstrate some generic techniques for sonifying 1D data:</u>

**First, import relevant modules:**

In [ ]:
%reload_ext autoreload 
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
from strauss.sonification import Sonification
from strauss.sources import Objects
from strauss import channels
from strauss.score import Score
from strauss.generator import Synthesizer
import IPython.display as ipd
import os
from scipy.interpolate import interp1d
import numpy as np

**Now, we construct some mock data!**

We use seeded random numbers to generate a mock 1D data set with features and noise:

In [ ]:
# seed the randoms...
np.random.seed(0)

# construct arrays of size N for x and y...
N = 300
x = np.linspace(0,1,N)
y = np.zeros(N)

# define a Gaussian function...
gauss = lambda x, m, s: np.exp(-(x-m)**2/s) 

# place some randomised gaussians...
for i in range(10):
    a,b,c = np.random.random(3)
    y += gauss(x, b, 1e-3*c) * a ** 3

# now add some noise and normalise
y += np.random.random(N) * y.mean()
y /= y.max()*1.2
y += 0.15


plt.plot(x,y)
plt.ylabel('Some dependent Variable')
plt.xlabel('Some independent Variable')

**Set up some universal sonification parameters and classes for the examples below**

For all examples we use the `Synthesizer` generator to create a 30 second, mono sonification.

In [ ]:
# specify audio system (e.g. mono, stereo, 5.1, ...)
system = "stereo"

# length of the sonification in s
length = 15.

# set up synth and turn on LP filter
generator = Synthesizer()
generator.load_preset('pitch_mapper')
generator.preset_details('pitch_mapper')

### <u>Example 1</u> &nbsp; **Pitch Mapping**

In [ ]:
notes = [["A2"]]
score =  Score(notes, length)

data = {'pitch':1.,
        'time_evo':x,
        'azimuth':(x*0.5+0.25) % 1,
        'polar':0.5,
        'pitch_shift':y**0.7}

# set up source
sources = Objects(data.keys())
sources.fromdict(data)
sources.apply_mapping_functions()

soni = Sonification(score, sources, generator, system)
soni.render()
soni.notebook_display()

### <u>Example 2</u> &nbsp; **Volume Mapping**

In [ ]:
notes = [["A2"]]
score =  Score(notes, length)

data = {'pitch':1.,
        'time_evo':x,
        'azimuth':(x*0.5+0.25) % 1,
        'polar':0.5,
        'volume':y**0.7}

# set up source
sources = Objects(data.keys())
sources.fromdict(data)
sources.apply_mapping_functions()

soni = Sonification(score, sources, generator, system)
soni.render()
soni.notebook_display()

### <u>Example 3</u> &nbsp; **Filter Cutoff Mapping**

In [ ]:
generator = Synthesizer()
generator.modify_preset({'filter':'on'})

In [ ]:
notes = [["C2","G2","C3","G3"]]
score =  Score(notes, length)

data = {'pitch':[0,1,2,3],
        'time_evo':[x]*4,
        'azimuth':[(x*0.5+0.25) % 1]*4,
        'polar':[0.5]*4,
        'cutoff':[y**0.8]*4}

# set up source
sources = Objects(data.keys())
sources.fromdict(data)
sources.apply_mapping_functions()

soni = Sonification(score, sources, generator, system)
soni.render()
soni.notebook_display()

In [ ]:
generator = Synthesizer()
generator.load_preset('windy')

In [ ]:
data = {'pitch':[0,1,2,3],
        'time_evo':[x],
        'azimuth':[(x*0.5+0.25) % 1],
        'polar':[0.5],
        'cutoff':[y**0.8]}
sources = Objects(data.keys())
sources.fromdict(data)
sources.apply_mapping_functions()

soni = Sonification(score, sources, generator, system)
soni.render()
soni.notebook_display()